In [74]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [75]:
# source
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

In [76]:
# load daily tracking
dailytracking = pd.read_excel(os.path.join(r'C:', user_credential, r'Dispatch files\Adherence & Daily Tracking\Daily Tracking.xlsx'), sheet_name='Daily Tracking', engine="openpyxl")

In [77]:
# subset Approved OT from daily tracking
approval = dailytracking[['EID', 'Session Date', 'Supervisor','OT','Approved Overtime']]
# convert date column to dt.date
approval['Session Date'] = pd.to_datetime(approval['Session Date'], format='mixed').dt.date

C:\Users\dinhhoang.nguyen.CONCENTRIX\AppData\Local\Temp\VSCodePortable-x64Temp\ipykernel_23216\1309633891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approval['Session Date'] = pd.to_datetime(approval['Session Date'], format='mixed').dt.date


In [78]:
# ramcoot, get everything except for NSA
ramcoot = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\ramcoot.json'))
ramcoot = ramcoot.loc[(ramcoot['OT Type'] != 'NSA') & (ramcoot['Status'] == 'Authorized')]

# calculate sum of OT authorised by agent by date
ramcoot = ramcoot.groupby(['employee_code', 'Date'], as_index=False)['Hours'].sum()

# convert ramcoot['Date'] to dt.date
ramcoot['Date'] = pd.to_datetime(ramcoot['Date'], format='mixed').dt.date

In [79]:
# create a frame
main_frame = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\schedule.json'))[['Emp ID', 'Date', 'LOB']]
# convert main_frame['Date'] to dt.date
main_frame['Date'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.date

In [80]:
# add approved OT into the frame
main_frame = pd.merge(main_frame, approval, left_on=['Emp ID', 'Date'], right_on=['EID', 'Session Date'], how='left')
# replace na with 0
main_frame[['OT', 'Approved Overtime']] = main_frame[['OT', 'Approved Overtime']].fillna(0)
# rename OT to Registered OT
main_frame = main_frame.rename(columns={'OT': 'Registered OT'})

In [81]:
# add OT submmitted on Ramco to main_frame
main_frame = pd.merge(main_frame, ramcoot, left_on=['Emp ID', 'Date'], right_on=['employee_code', 'Date'], how='left')
# replace na with 0
main_frame['Hours'] = main_frame['Hours'].fillna(0)
# rename OT to Registered OT
main_frame = main_frame.rename(columns={'Hours': 'Ramco OT'})

In [82]:
#drop abundant columns: EID, employee_code, Session Date
main_frame =main_frame.drop(columns={'EID', 'employee_code', 'Session Date'})
# add month
main_frame['Month'] = pd.to_datetime(main_frame['Date'], format='mixed').dt.strftime('%Y-%m')
# add discrepancy
main_frame['Discrepancies'] = main_frame['Approved Overtime'] - main_frame['Ramco OT']
# add discrepancy type
main_frame['Discre_type'] = main_frame.apply(lambda x: 'Valid' if x['Discrepancies'] == 0 else 'Over-submitted' if x['Discrepancies']<0 else 'Under-submitted', axis=1)
# remove rows where Supervisor is not na
main_frame = main_frame.loc[main_frame['Supervisor'].notnull()]
# add a column to help TLs to know who is incharge of adjust the ramco submission, depends on the date submit and current date
main_frame['#submission_to_current'] = (dt.today() - pd.to_datetime(main_frame['Date'], format='mixed')).dt.days.astype('int64')
main_frame['person_to_action'] = main_frame.apply(lambda x: 'no_action' if x['Discre_type'] == 'Valid' else '<= 7 days /nTL' if x['Discre_type'] != '<=12 days /nValid' and x['#submission_to_current']<= 7 else 'OM' if x['Discre_type'] != 'Valid' and x['#submission_to_current']<= 12 else '>12 days /nHR', axis=1)
summary = main_frame.loc[main_frame['person_to_action']!= 'no_action'].pivot_table(index=['Supervisor', 'Month'], columns=['person_to_action'],values='Emp ID', aggfunc='count').reset_index()

In [93]:
with pd.ExcelWriter(os.path.join(r'C:', user_credential, r'Dispatch files\Validate OT\OTMismatches.xlsx'), mode="a", engine="openpyxl",if_sheet_exists="replace") as writer:
    main_frame.to_excel(writer, sheet_name='Details', index=False)